In [ ]:
import os
import cv2

#  paths
dataset_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face"
train_images_dir = os.path.join(dataset_path, "WIDER_train", "images")
val_images_dir = os.path.join(dataset_path, "WIDER_val", "images")
train_anno_file = os.path.join(dataset_path, "WIDER_train", "wider_face_train_bbx_gt.txt")
val_anno_file = os.path.join(dataset_path, "WIDER_val", "wider_face_val_bbx_gt.txt")

train_labels_dir = os.path.join(dataset_path, "WIDER_train", "labels2")
val_labels_dir = os.path.join(dataset_path, "WIDER_val", "labels2")
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

def convert_to_yolo(images_dir, anno_file, labels_dir):
    with open(anno_file, "r") as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        # is the line an image filename
        if ".jpg" in line:
            image_rel_path = line
            image_abs_path = os.path.join(images_dir, image_rel_path)
            label_file = os.path.join(labels_dir, image_rel_path.replace(".jpg", ".txt"))
            os.makedirs(os.path.dirname(label_file), exist_ok=True)

            if not os.path.exists(image_abs_path):
                print(f"Image not found: {image_abs_path}")
                i += 1
                continue
            img = cv2.imread(image_abs_path)
            img_h, img_w = img.shape[:2]

            #number of faces
            num_faces = int(lines[i + 1].strip())
            annotations = lines[i + 2 : i + 2 + num_faces]

            with open(label_file, "w") as lf:
                for ann in annotations:
                    ann = ann.strip().split()
                    x, y, w, h = map(int, ann[:4])

                    #  YOLO format
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    width = w / img_w
                    height = h / img_h

                    lf.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

            i += 2 + num_faces
        else:
            i += 1

#  train and val datasets
convert_to_yolo(train_images_dir, train_anno_file, train_labels_dir)
convert_to_yolo(val_images_dir, val_anno_file, val_labels_dir)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  #  YOLOv8-N pre-trained weights


In [ ]:
model.train(
    data=r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\data.yaml",  
    epochs=50,           
    imgsz=640,                 
    batch=16                   
)

Ultralytics 8.3.40  Python-3.12.3 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

train: Scanning C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_train\labels\1--Handshaking.cache... 12420 images, 3 backgrounds, 1 corrupt: 100%|██████████| 12420/12420 [00:00<?, ?it/s]

train: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_train\images\2--Demonstration\2_Demonstration_Protesters_2_231.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_train\images\37--Soccer\37_Soccer_Soccer_37_851.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_train\images\54--Rescue\54_Rescue_rescuepeople_54_29.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0254]
train: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_train\images\7--Cheering\7_Cheering_Cheering_7_17.jpg: 1 duplicate labels removed



val: Scanning C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_val\labels\1--Handshaking.cache... 3111 images, 0 backgrounds, 1 corrupt: 100%|██████████| 3111/3111 [00:00<?, ?it/s]

val: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_val\images\21--Festival\21_Festival_Festival_21_604.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\wider_face\WIDER_val\images\39--Ice_Skating\39_Ice_Skating_iceskiing_39_583.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.002]


...

      50/50      17.6G       1.31     0.6114     0.9642         17        640: 100%|██████████| 777/777 [01:18<00:00,  9.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:12<00:00,  7.57it/s]


                   all       3110      35727      0.843      0.578      0.655      0.364

50 epochs completed in 1.832 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.40  Python-3.12.3 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:12<00:00,  7.98it/s]


                   all       3110      35727      0.844      0.576      0.655      0.364
Speed: 0.2ms preprocess, 0.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train9


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021F27AF9160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
#TESTING
from ultralytics import YOLO
import cv2

model = YOLO(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Notebooks\runs\detect\train9\weights\best.pt')

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

SCALE_FACTOR = 1.25 

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    results = model.predict(source=frame, conf=0.25, imgsz=640, show=False)

    annotated_frame = frame.copy()
    for result in results:
        for box, conf in zip(result.boxes.xyxy, result.boxes.conf): 
            x1, y1, x2, y2 = box.tolist()
            confidence = float(conf)  
            width = x2 - x1
            height = y2 - y1

            new_width = width * SCALE_FACTOR
            new_height = height * SCALE_FACTOR
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            x1_new = max(0, x_center - new_width / 2)
            y1_new = max(0, y_center - new_height / 2)
            x2_new = min(frame.shape[1], x_center + new_width / 2)
            y2_new = min(frame.shape[0], y_center + new_height / 2)

            cv2.rectangle(annotated_frame, (int(x1_new), int(y1_new)), (int(x2_new), int(y2_new)), (0, 255, 0), 2)


            text = f"{confidence:.2f}"
            cv2.putText(
                annotated_frame, text, (int(x1_new), int(y1_new) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("YOLO Inference", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 face, 75.0ms
Speed: 4.0ms preprocess, 75.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 57.5ms
Speed: 1.5ms preprocess, 57.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 53.5ms
Speed: 1.5ms preprocess, 53.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 55.0ms
Speed: 1.0ms preprocess, 55.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 60.5ms
Speed: 1.5ms preprocess, 60.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 59.0ms
Speed: 0.5ms preprocess, 59.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 faces, 62.5ms
Speed: 1.0ms preprocess, 62.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO(r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Notebooks\runs\detect\train9\weights\last.pt") 

image = cv2.imread(r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\images\11.jpg")
results = model.predict(image)

for result in results[0].boxes.data.tolist():
    x1, y1, x2, y2, confidence, class_id = result
    print(f"Bounding Box: [{x1}, {y1}, {x2}, {y2}], Confidence: {confidence}, Class: {model.names[int(class_id)]}") #testing bbox and conf to see the format


0: 640x640 1 face, 39.9ms
Speed: 1.9ms preprocess, 39.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Bounding Box: [145.15359497070312, 99.60334777832031, 292.3818664550781, 312.666748046875], Confidence: 0.8789143562316895, Class: face
